# Installation and Setup

In [ ]:
# download the corpus for nltk and spacy's english language model
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install pyLDAvis==3.2.2

     |████████████████████████████████| 1.7MB 3.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135618 sha256=0ed7dae71701208e7057533162dba6296f0cbad165e226c875958610c08c297f
  Stored in directory: /root/.cache/pip/wheels/74/df/b6/97234c8446a43be05c9a8687ee0db1f1b5ade5f27729187eae
Successfully built pyLDAvis


# importing packages

In [ ]:
import re
import numpy as np
import pandas as pd

# GENSIM
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import spacy

# importing data

In [ ]:
# importing newsgroup data
from sklearn import datasets
emaildata = datasets.fetch_20newsgroups()
data = emaildata.data

In [ ]:
len(data)

In [ ]:
print(data[100])

From: (Tsung-Kun Chen) Subject: ** Software forsale (lots) ** Nntp-Posting-Host: magnusug.magnus.acs.ohio-state.edu Organization: The Ohio State University **** This is a post for my friend, You can either call **** **** him J.K Lee (614)791-0748 or Drop me a mail **** Distribution: usa Lines: 39 1. Software publishing SuperBase 4 windows v.1.3 --->$80 2. OCR System ReadRight v.3.1 for Windows --->$65 3. OCR System ReadRight v.2.01 for DOS --->$65 4. Unregistered Zortech 32 bit C++ Compiler v.3.1 --->$ 250 with Multiscope windows Debugger, WhiteWater Resource Toolkit, Library Source Code 5. Glockenspiel/ImageSoft Commonview 2 Windows Applications Framework for Borland C++ --->$70 6. Spontaneous Assembly Library With Source Code --->$50 7. Microsoft Macro Assembly 6.0 --->$50 8. Microsoft Windows v.3.1 SDK Documentation --->$125 9. Microsoft FoxPro V.2.0 --->$75 10. WordPerfect 5.0 Developers Toolkit --->$20 11. Kedwell Software DataBoss v.3.5 C Code Generator --->$100 12. Kedwell Insta

# Data Cleaning

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

## Tokenize words and Clean-up text

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


# Creating Bigram and Trigram Models

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


# Remove Stopwords, Make Bigrams and Lemmatize

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


# Topic Modeling

In [ ]:
# Create Dictionary
dictionary = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# View the topics in LDA model

In [ ]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.051*"report" + 0.027*"black" + 0.020*"fire" + 0.020*"white" + 0.016*"trial" + 0.016*"cover" + 0.015*"medium" + 0.013*"vote" + 0.012*"minor" + 0.012*"title"'), (1, '0.021*"god" + 0.020*"accept" + 0.016*"member" + 0.015*"man" + 0.014*"israeli" + 0.014*"season" + 0.012*"publish" + 0.012*"lebanese" + 0.012*"jewish" + 0.011*"brain"'), (2, '0.017*"package" + 0.016*"press" + 0.015*"item" + 0.015*"break" + 0.011*"level" + 0.010*"edge" + 0.009*"hole" + 0.007*"eye" + 0.007*"contribute" + 0.007*"equipment"'), (3, '0.025*"pc" + 0.022*"contain" + 0.020*"input" + 0.020*"reality" + 0.017*"picture" + 0.016*"object" + 0.016*"level" + 0.015*"box" + 0.015*"quality" + 0.013*"greek"'), (4, '0.089*"ax" + 0.076*"max" + 0.032*"space" + 0.021*"launch" + 0.018*"di_di" + 0.017*"orbit" + 0.016*"sphere" + 0.015*"satellite" + 0.014*"plane" + 0.014*"mission"'), (5, '0.019*"people" + 0.017*"kill" + 0.015*"child" + 0.015*"government" + 0.012*"attack" + 0.012*"year" + 0.012*"die" + 0.011*"country" + 0.010*"say"

In [ ]:
lda_model.print_topics()

[(0,
  '0.051*"report" + 0.027*"black" + 0.020*"fire" + 0.020*"white" + 0.016*"trial" + 0.016*"cover" + 0.015*"medium" + 0.013*"vote" + 0.012*"minor" + 0.012*"title"'),
 (1,
  '0.021*"god" + 0.020*"accept" + 0.016*"member" + 0.015*"man" + 0.014*"israeli" + 0.014*"season" + 0.012*"publish" + 0.012*"lebanese" + 0.012*"jewish" + 0.011*"brain"'),
 (2,
  '0.017*"package" + 0.016*"press" + 0.015*"item" + 0.015*"break" + 0.011*"level" + 0.010*"edge" + 0.009*"hole" + 0.007*"eye" + 0.007*"contribute" + 0.007*"equipment"'),
 (3,
  '0.025*"pc" + 0.022*"contain" + 0.020*"input" + 0.020*"reality" + 0.017*"picture" + 0.016*"object" + 0.016*"level" + 0.015*"box" + 0.015*"quality" + 0.013*"greek"'),
 (4,
  '0.089*"ax" + 0.076*"max" + 0.032*"space" + 0.021*"launch" + 0.018*"di_di" + 0.017*"orbit" + 0.016*"sphere" + 0.015*"satellite" + 0.014*"plane" + 0.014*"mission"'),
 (5,
  '0.019*"people" + 0.017*"kill" + 0.015*"child" + 0.015*"government" + 0.012*"attack" + 0.012*"year" + 0.012*"die" + 0.011*"count

# Compute Model Coherence Score

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics-keywords

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.338658  0.015795       1        1  24.176076
7     -0.310554 -0.102087       2        1  15.732083
17    -0.229563  0.076675       3        1  12.436313
5     -0.141188 -0.192220       4        1   5.442055
9     -0.050182  0.253796       5        1   4.782111
6     -0.022123  0.285904       6        1   3.871352
19     0.020493 -0.012533       7        1   3.301804
8      0.040339 -0.000898       8        1   3.247854
18    -0.051658 -0.014655       9        1   3.241523
1      0.023278 -0.129477      10        1   3.160816
12     0.062984  0.064512      11        1   2.908647
2      0.114210 -0.004330      12        1   2.658866
16     0.055807 -0.140878      13        1   2.352411
10     0.111575 -0.040583      14        1   2.160203
3      0.126656  0.020912      15        1   2.000029
14     0.095916 -0.076025      16        1   1.896464
4      0.131738  0.017564      17        1   1.752229
11     0.108804  0.029916      18        1   1.742686
0      0.121754 -0.036484      19        1   1.631909
15     0.130372 -0.014904      20        1   1.504570, topic_info=           Term          Freq         Total Category  logprob  loglift
19         line  11975.000000  11975.000000  Default  30.0000  30.0000
197       write  10060.000000  10060.000000  Default  29.0000  29.0000
174       would  12046.000000  12046.000000  Default  28.0000  28.0000
139      people   5647.000000   5647.000000  Default  27.0000  27.0000
177     article   6479.000000   6479.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
3369       seat    121.906449    145.087899  Topic20  -4.9370   4.0226
756    purchase    167.274638    311.113662  Topic20  -4.6207   3.5761
521        lead    254.004496    862.104624  Topic20  -4.2029   2.9746
205   condition    153.886640    421.639776  Topic20  -4.7041   3.1887
989        stop    136.921554    898.996100  Topic20  -4.8209   2.3148

[945 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
4621      6  0.991735  accelerator
1211      6  0.015122       accept
1211     10  0.984293       accept
81        3  0.007225       access
81        6  0.382922       access
...     ...       ...          ...
40        8  0.006502         year
40        9  0.371745         year
40       16  0.003110         year
720       9  0.990713    yesterday
570      14  0.998419        young

[1424 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 8, 18, 6, 10, 7, 20, 9, 19, 2, 13, 3, 17, 11, 4, 15, 5, 12, 1, 16])

# LDA Mallet Model

You can achive better performance with LDA Mallet model

In [ ]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = r"D:\AI\jpmc_mumbai\mallet-2.0.8\mallet-2.0.8\bin\mallet" # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=dictionary)


In [ ]:
# Show Topics
print(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

# finding the optimal number of topics for LDA?

Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. Picking an even higher value can sometimes provide more granular sub-topics.

If you see the same keywords being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()